In [7]:
from ultralytics import YOLO
import cv2
import os
from glob import glob

In [19]:
VIDEO_GLOB = os.path.join("../test", "videos", "*.mp4")
VEHICLE_CLASSES = {2, 3, 5, 7}
CONF_THRESH = 0.5

vehicle_model = YOLO("../yolov8n.pt")
plate_model = YOLO("../detection-plate/runs/detect/train/weights/best.pt")

video_glob = os.path.join("../test", "videos", "*.mp4")
video_files = sorted(glob(video_glob))
if not video_files:
    raise FileNotFoundError(f"No videos found with pattern: {video_glob}")

first_video = video_files[0]
print("Opening:", first_video)

Opening: ../test/videos/test-video1.mp4


In [20]:
def process_video(
    video_path: str,
    vehicle_model: YOLO,
    plate_model: YOLO,
    output_path: str | None = None,
):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {video_path}")

    writer = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        fps = cap.get(cv2.CAP_PROP_FPS) or 20
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 1) detect & track vehicles
        v_res = vehicle_model.track(frame, persist=True)[0]
        for *box, track_id, score, cls in v_res.boxes.data.tolist():
            if score < CONF_THRESH or int(cls) not in VEHICLE_CLASSES:
                continue

            x1, y1, x2, y2 = map(int, box)
            roi = frame[y1:y2, x1:x2]

            # 2) detect plates inside that vehicle ROI
            p_res = plate_model(roi)[0]
            for px1, py1, px2, py2, p_score, _ in p_res.boxes.data.tolist():
                if p_score < CONF_THRESH:
                    continue

                # map back to full frame coords
                X1, Y1 = x1 + int(px1), y1 + int(py1)
                X2, Y2 = x1 + int(px2), y1 + int(py2)
                cv2.rectangle(frame, (X1, Y1), (X2, Y2), (0, 255, 0), 2)

        # 3) draw vehicle boxes/tracks if you like
        for *box, track_id, score, cls in v_res.boxes.data.tolist():
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(
                frame,
                f"ID:{int(track_id)}",
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 255),
                1,
            )

        # 4) write out / show
        if writer:
            writer.write(frame)
        else:
            cv2.imshow("pipeline", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    cap.release()
    if writer:
        writer.release()
    else:
        cv2.destroyAllWindows()

In [21]:
videos = sorted(glob(VIDEO_GLOB))
if not videos:
    raise FileNotFoundError(f"No .mp4s in {VIDEO_GLOB}")
first_vid = videos[0]
print("Processing:", first_vid)

Processing: ../test/videos/test-video1.mp4


In [23]:
device = "cuda" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "cpu"
out_path="./video-results"

process_video(first_vid, vehicle_model, plate_model, output_path=out_path)
print("Done ▶️", out_path)

[ WARN:0@760.144] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'





0: 384x640 21 cars, 1 bus, 2 trucks, 23.4ms
Speed: 1.5ms preprocess, 23.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 608x640 (no detections), 39.3ms
Speed: 1.6ms preprocess, 39.3ms inference, 0.2ms postprocess per image at shape (1, 3, 608, 640)

0: 544x640 (no detections), 62.0ms
Speed: 1.1ms preprocess, 62.0ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)

0: 608x640 1 License_Plate, 37.2ms
Speed: 1.5ms preprocess, 37.2ms inference, 0.3ms postprocess per image at shape (1, 3, 608, 640)

0: 640x640 (no detections), 38.9ms
Speed: 1.3ms preprocess, 38.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 License_Plate, 38.2ms
Speed: 1.5ms preprocess, 38.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 544x640 1 License_Plate, 33.5ms
Speed: 1.2ms preprocess, 33.5ms inference, 0.3ms postprocess per image at shape (1, 3, 544, 640)

0: 192x640 (no detections), 15.2ms
Speed: 0.6ms preprocess,

KeyboardInterrupt: 